## Chatbot

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nurul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import json
import pandas as pd

# Load the dataset from a JSON file
with open('dataset2.json') as f:
    data = json.load(f)

# Extract input, responses, and tags
tags = []
inputs = []
responses = {}
for intent in data['intents']: 
    responses[intent['tag']] = intent['responses']
    for line in intent['patterns']:
        inputs.append(line)
        tags.append(intent['tag'])

# Create a DataFrame
data_df = pd.DataFrame({"inputs": inputs, "tags": tags})


In [3]:
# Remove punctuation and convert to lowercase
lemmatizer = WordNetLemmatizer()
data_df['inputs'] = data_df['inputs'].apply(lambda wrd: ' '.join([lemmatizer.lemmatize(word.lower()) for word in wrd.split() if word not in string.punctuation]))


In [4]:
print(data_df)

                                                inputs  \
0                                           menstruasi   
1                                  apa itu menstruasi?   
2                                    siklus menstruasi   
3                berapa lama siklus menstruasi normal?   
4                          kapan haid biasanya datang?   
..                                                 ...   
259  bagaimana cara mengatasi polycystic ovary synd...   
260  apakah polycystic ovary syndrome (pcos) dapat ...   
261                           menstruasi tidak teratur   
262     apa yang menyebabkan menstruasi tidak teratur?   
263  bagaimana cara mengatasi menstruasi tidak tera...   

                                 tags  
0                          Menstruasi  
1                          Menstruasi  
2                              siklus  
3                              siklus  
4                              siklus  
..                                ...  
259  Polycystic Ovary S

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder


# Tokenize the data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data_df['inputs'])
train_sequences = tokenizer.texts_to_sequences(data_df['inputs'])

# Apply padding
x_train = pad_sequences(train_sequences)

# Encode the outputs
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(data_df['tags'])

# Define input shape
input_shape = x_train.shape[1]
print("Input shape:", input_shape)

# Define vocabulary size
vocabulary_size = len(tokenizer.word_index)
print("Number of unique words:", vocabulary_size)

# Define output length
output_length = len(label_encoder.classes_)
print("Output length:", output_length)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size + 1, 10, input_length=input_shape),
    tf.keras.layers.LSTM(10, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(output_length, activation='softmax')
])


Input shape: 14
Number of unique words: 243
Output length: 62


In [5]:

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
num_epochs = 200
history = model.fit(x_train, y_train, epochs=num_epochs)

# Define function to predict answer
def predict_answer(model, tokenizer, label_encoder, question):
    question = ''.join([letters.lower() for letters in question if letters not in string.punctuation])
    texts_p = [question]
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)
    output = model.predict(prediction_input)
    output = output.argmax()
    response_tag = label_encoder.inverse_transform([output])[0]
    return response_tag


Epoch 1/200
9/9 [==============================] - 4s 11ms/step - loss: 4.1279 - accuracy: 0.0153
Epoch 2/200
9/9 [==============================] - 0s 10ms/step - loss: 4.1235 - accuracy: 0.0382
Epoch 3/200
9/9 [==============================] - 0s 11ms/step - loss: 4.1199 - accuracy: 0.0382
Epoch 4/200
9/9 [==============================] - 0s 11ms/step - loss: 4.1152 - accuracy: 0.0344
Epoch 5/200
9/9 [==============================] - 0s 11ms/step - loss: 4.1088 - accuracy: 0.0344
Epoch 6/200
9/9 [==============================] - 0s 12ms/step - loss: 4.0999 - accuracy: 0.0344
Epoch 7/200
9/9 [==============================] - 0s 10ms/step - loss: 4.0886 - accuracy: 0.0344
Epoch 8/200
9/9 [==============================] - 0s 10ms/step - loss: 4.0703 - accuracy: 0.0344
Epoch 9/200
9/9 [==============================] - 0s 12ms/step - loss: 4.0485 - accuracy: 0.0344
Epoch 10/200
9/9 [==============================] - 0s 11ms/step - loss: 4.0252 - accuracy: 0.0344
Epoch 11/200
9/9 [=

In [6]:
import random 
# Start chatbot
while True:
    user_input = input('You: ')
    if user_input.lower() == 'goodbye':
        print("Chatbot: Goodbye!")
        break
    response_tag = predict_answer(model, tokenizer, label_encoder, user_input)
    print("Chatbot:", random.choice(responses.get(response_tag, ["I'm sorry, I don't understand that."])))

1/1 [==============================] - 1s 784ms/step
Chatbot: Siklus menstruasi adalah periode waktu antara dua periode haid. Untuk menghitungnya, Anda dapat mulai dari hari pertama haid Anda sebagai hari pertama siklus.
1/1 [==============================] - 0s 49ms/step
Chatbot: Anda bisa mencoba minum obat pereda nyeri atau menggunakan bantalan hangat untuk meredakan sakit perut.
1/1 [==============================] - 0s 42ms/step
Chatbot: Beberapa gejala umum menstruasi meliputi sakit perut, mual, sakit kepala, dan perubahan suasana hati. Ini adalah hal yang normal selama menstruasi.
1/1 [==============================] - 0s 48ms/step
Chatbot: Anda bisa mencoba minum obat pereda nyeri atau menggunakan bantalan hangat untuk meredakan sakit perut.
1/1 [==============================] - 0s 42ms/step
Chatbot: Selama menstruasi, tubuh Anda melepaskan lapisan rahim yang tidak diperlukan. Ini adalah proses alami yang terjadi setiap bulan.
1/1 [==============================] - 0s 43ms/ste